In [ ]:
import requests
import pandas as pd
import time

# Konfigurasi API ThingSpeak
channel_id = 2326256
read_api_key = 'BKIJNOYDM205J2HG'
url = f'https://api.thingspeak.com/channels/{channel_id}/feeds.json'

# Jumlah maksimum data per permintaan (maksimal 8000 menurut dokumentasi ThingSpeak)
max_results_per_page = 8000

# Inisialisasi variabel untuk menampung semua data
all_data = []

# Parameter paginasi
page = 1
total_results = 0

# Loop untuk mengambil data dari semua halaman
while True:
    # Parameter untuk permintaan API
    params = {
        'api_key': read_api_key,
        'results': max_results_per_page,
        'offset': (page - 1) * max_results_per_page
    }

    # Mengirim permintaan GET ke ThingSpeak
    response = requests.get(url, params=params)

    # Cek apakah permintaan berhasil
    if response.status_code == 200:
        # Mengambil data feed dalam format JSON
        data = response.json()
        feeds = data['feeds']

        # Jika tidak ada data lagi, keluar dari loop
        if not feeds:
            break

        # Tambahkan data feed ke list all_data
        all_data.extend(feeds)

        # Tampilkan jumlah data yang sudah diambil
        total_results += len(feeds)
        print(f"Fetched {total_results} total records so far...")

        # Tunggu sejenak agar tidak membanjiri server ThingSpeak
        time.sleep(1)

        # Lanjutkan ke halaman berikutnya
        page += 1
    else:
        print(f"Failed to fetch data on page {page}. HTTP Status code: {response.status_code}")
        break

# Membuat DataFrame Pandas dari semua data yang dikumpulkan
df = pd.DataFrame(all_data)

# Pilih kolom-kolom yang ingin disimpan, misalnya waktu dan field1 hingga field8
df = df[['created_at', 'field1', 'field2', 'field3', 'field4', 'field8']]

# Ganti nama kolom untuk lebih deskriptif
df.columns = ['Time', 'Suhu', 'Kelembapan', 'Tekanan', 'Embun', 'Volt']

# Simpan DataFrame ke file CSV
df.to_csv('Cuaca_All_Data.csv', index=False)

# Tampilkan DataFrame untuk pengecekan
df.tail(10)


Fetched 8000 total records so far...
Fetched 16000 total records so far...
Fetched 24000 total records so far...
Fetched 32000 total records so far...
Fetched 40000 total records so far...
Fetched 48000 total records so far...
Fetched 56000 total records so far...
Fetched 64000 total records so far...
Fetched 72000 total records so far...
Fetched 80000 total records so far...
Fetched 88000 total records so far...
Fetched 96000 total records so far...
Fetched 104000 total records so far...
Fetched 112000 total records so far...
Fetched 120000 total records so far...
Fetched 128000 total records so far...
Fetched 136000 total records so far...
Fetched 144000 total records so far...
Fetched 152000 total records so far...
Fetched 160000 total records so far...
Fetched 168000 total records so far...
Fetched 176000 total records so far...
Fetched 184000 total records so far...
Fetched 192000 total records so far...
Fetched 200000 total records so far...
Fetched 208000 total records so far...

In [4]:
dataharian = (lambda: _deepnote_execute_sql('SELECT *\nFROM \'Cuaca.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT *\nFROM \'Cuaca.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
dataharian

,Time,Suhu,Kelembapan,Tekanan,Embun,Volt
0,2024-09-29 17:59:52,26.36,92.39,1011.36,25.03,4.04
1,2024-09-29 18:00:52,26.35,92.52,1011.39,25.04,4.04
2,2024-09-29 18:01:52,26.33,92.38,1011.38,24.99,4.04
3,2024-09-29 18:02:52,26.35,92.36,1011.44,25.01,4.04
4,2024-09-29 18:03:51,26.33,92.27,1011.44,24.97,4.04
...,...,...,...,...,...,...
7995,2024-10-05 10:24:22,28.98,82.66,1010.20,25.73,4.06
7996,2024-10-05 10:25:22,28.99,82.68,1010.20,25.74,4.06
7997,2024-10-05 10:26:22,28.96,82.75,1010.24,25.73,4.06
7998,2024-10-05 10:27:22,28.96,82.74,1010.22,25.73,4.09


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>